Baigus darbą visu failus suarchyvuoti ir atsiųsti email tomas.rasymas@gmail.com su savo vardu ir pavarde.

Kodą būtina komentuoti.

# 1 užduotis

Sukurti modelius:

**Shop (parduotuvė)**, lentelės pavadinimas - shops:
* id - sveikas skaičius, pirminis raktas
* name - tekstas 40 simbolių ilgio, privalomas (nėra tuščias)
* address - tekstas 100 simbolių ilgio
* items - ryšys į Item modelį

**Item (prekė)**, lentelės pavadinimas - items:
* id - sveikas skaičius, pirminis raktas
* barcode - tekstas 32 simbolių ilgio, unikalus
* name - tekstas 40 simbolių ilgio, privalomas (nėra tuščias)
* description - tekstas 200 simbolių ilgio, numatyta reikšmė - EMPTY
* unit_price - dešimtainis skaičius su tikslumu (10, 2), privalomas (nėra tuščias), numatyta reikšmė 1.00
* created_at - data ir laikas (DateTime), numatyta reikšmė - įrašo sukūrimo data ir laikas
* shop_id - sveikas skaičius, ForeignKey į shops.id
* shop - ryšys į Shop modelį
* components - ryšys į Component modelį

**Component (komponentas)**, lentelės pavadinimas - components:
* id - sveikas skaičius, pirminis raktas
* name - tekstas 20 simbolių ilgio
* quantity - dešimtainis skaičius su tikslumu (10, 2), numatyta reikšmė 1.00
* item_id - sveikas skaičius, ForeignKey į items.id
* item - ryšys į Item modelį

Viena parduotuvė (shop) turi daug prekių (item), viena prekė (item) yra sudaryta iš daug komponentų.

# 2 užduotis

Sukurti įrašus modeliuose:

**Shop:**
* name='IKI', address='Kaunas, Iki gatvė 1'
* name='MAXIMA', address='Kaunas, Maksima gatvė 2'

**Item:** 
* **IKI shop**
  * barcode='112233112233', name='Žemaičių duona', unit_price=1.55
  * barcode='33333222111', description='Pienas iš Žemaitijos', name='Žemaičių pienas', unit_price=2.69
* **MAXIMA shop**
  * barcode='99898989898', name='Aukštaičių duona', unit_price=1.65
  * barcode='99919191991', description='Pienas iš Aukštaitijos', name='Aukštaičių pienas', unit_price=2.99
 
**Component**
* **IKI Žemaičių duona**
  * name='Miltai', quantity=1.50
  * name='Vanduo', quantity=1.00
* **IKI Žemaičių pienas**
  * name='Pienas', quantity=1.00
* **MAXIMA Aukštaičių duona**
  * name='Miltai', quantity=1.60
  * name='Vanduo', quantity=1.10
* **MAXIMA Aukštaičių pienas**
  * name='Pienas', quantity=1.10

# 3 užduotis

* Pakeisti 'IKI' parduotuvės, 'Žemaičių duonos' komponento 'Vandens' kiekį (quantity) iš 1.00 į 1.45.
* Ištrinti 'MAXIMA' parduotuvės, 'Aukštaičių pieno' komponentą 'Pienas'.

# 4 užduotis

Atspausdinti visų parduotuvių prekes, bei tų prekių komponentus.

# 5 užduotis

Sukurti užklausas:
* Atrinkti prekes, kurios turi susietų komponentų
* Atrinkti prekes, kurių pavadinime yra tekstas 'ien'
* Suskaičiuoti iš kiek komponentų sudaryta kiekviena prekė
* Suskaičiuoti kiekvienos prekės komponentų kiekį (quantity)
* Savo nuožiūra suformuoti pasirinktų duomenų užklausą, bei ją aprašyti.